In [ ]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = False # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
    model['W2'] = np.random.randn(H) / np.sqrt(H)

grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
    if render: env.render()

    # preprocess the observation, set input to network to be difference image
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
        episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
        epx = np.vstack(xs)
        eph = np.vstack(hs)
        epdlogp = np.vstack(dlogps)
        epr = np.vstack(drs)
        xs,hs,dlogps,drs = [],[],[],[] # reset array memory

        # compute the discounted reward backwards through time
        discounted_epr = discount_rewards(epr)
        # standardize the rewards to be unit normal (helps control the gradient estimator variance)
        discounted_epr -= np.mean(discounted_epr)
        discounted_epr /= np.std(discounted_epr)

        epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
        grad = policy_backward(eph, epdlogp)
        for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

    # perform rmsprop parameter update every batch_size episodes
        if episode_number % batch_size == 0:
            for k,v in model.iteritems():
                g = grad_buffer[k] # gradient
                rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
                model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
                grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
    if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env
    prev_x = None

    if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
        print ('ep %d: game finished, reward: %f' % (episode_number, reward)) + ('' if reward == -1 else ' !!!!!!!!')

### Mountain Car

In [1]:
import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from collections import deque

class DQN:
    def __init__(self, env):
        self.env     = env
        self.memory  = deque(maxlen=2000)
        
        self.gamma = 0.85
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.005
        self.tau = .125

        self.model        = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        state_shape  = self.env.observation_space.shape
        model.add(Dense(24, input_dim=state_shape[0], activation="relu"))
        model.add(Dense(48, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.env.action_space.n))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        return np.argmax(self.model.predict(state)[0])

    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = 32
        if len(self.memory) < batch_size: 
            return

        samples = random.sample(self.memory, batch_size)
        for sample in samples:
            state, action, reward, new_state, done = sample
            target = self.target_model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.target_model.predict(new_state)[0])
                target[0][action] = reward + Q_future * self.gamma
            self.model.fit(state, target, epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)

def main():
    env     = gym.make("MountainCar-v0")
    gamma   = 0.9
    epsilon = .95

    trials  = 1000
    trial_len = 500

    # updateTargetNetwork = 1000
    dqn_agent = DQN(env=env)
    steps = []
    for trial in range(trials):
        cur_state = env.reset().reshape(1,2)
        for step in range(trial_len):
            env.render()
            action = dqn_agent.act(cur_state)
            new_state, reward, done, _ = env.step(action)

            # reward = reward if not done else -20
            new_state = new_state.reshape(1,2)
            dqn_agent.remember(cur_state, action, reward, new_state, done)
            
            print(len(dqn_agent.memory))
            dqn_agent.replay()       # internally iterates default (prediction) model
            dqn_agent.target_train() # iterates target model

            cur_state = new_state
            if done:
                break
        if step >= 199:
            print("Not Reached".format(trial))
            if step % 10 == 0:
                dqn_agent.save_model("trial-{}.model".format(trial))
        else:
            print("Completed in {} trials".format(trial))
            dqn_agent.save_model("success.model")
            break

if __name__ == "__main__":
    main()

Using TensorFlow backend.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Failed to complete in trial 0
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
27

1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
Failed to complete in trial 9


KeyboardInterrupt: 

### Pendulum

In [1]:
import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from collections import deque
import os

Using TensorFlow backend.


In [2]:
env = gym.make('Acrobot-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1000
output_dir = 'model_output/Acrobat/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
class AcrobatArm():
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = 0.001
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.memory = deque(maxlen = 2000)
        self.model = self.build_model()
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(32, activation = 'relu', input_dim = self.state_size))
        model.add(Dense(32, activation = 'relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def train(self, batch_size):
        if len(agent.memory) >= batch_size:
            mini_batch = random.sample(self.memory, batch_size)

            if done:
                target = reward
            else:
                target = reward + np.max(self.gamma*self.model.predict(next_state))

            y = self.model.predict(state)
            y[0][action] = target

            self.model.fit(state, y, epochs=1, verbose=0)

        
    def act(self, state):
        if self.epsilon > np.random.randn():
            return env.action_space.sample()
        pred = self.model.predict(state)
        return np.argmax(pred[0])
    
    def save(self, name): 
        self.model.save_weights(name)

In [4]:
agent = AcrobatArm(state_size, action_size)

In [5]:
for epi in range(n_episodes):
    state = env.reset().reshape(1, state_size)
    
    done = False
    time= 0
    reward_in_epi = []
    while not done:
        env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward_in_epi.append(reward)
        
        #reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        agent.train(batch_size)
        state = next_state
        if done: 
            if time >=199:
                print("episode: {}/{}, TReward {}, e: {:.2}".format(epi, n_episodes-1, np.mean(reward_in_epi), agent.epsilon))
            else:
                print('-'*20 + 'Reached  episode: {}/{} TReward {}, e: {:.2}'.format(epi, n_episodes-1, np.mean(reward_in_epi), agent.epsilon))
        
        time+=1

        
    if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay
            
    if epi%50 == 0:
        agent.save(output_dir + "weights_" + '{:04d}'.format(epi) + ".hdf5")

episode: 0/999, TReward -1.0, e: 1.0
episode: 1/999, TReward -0.9970674486803519, e: 0.99
episode: 2/999, TReward -1.0, e: 0.99
episode: 3/999, TReward -1.0, e: 0.99
episode: 4/999, TReward -1.0, e: 0.98
episode: 5/999, TReward -1.0, e: 0.98
episode: 6/999, TReward -0.997229916897507, e: 0.97
episode: 7/999, TReward -1.0, e: 0.97
episode: 8/999, TReward -1.0, e: 0.96
episode: 9/999, TReward -1.0, e: 0.96
episode: 10/999, TReward -1.0, e: 0.95
episode: 11/999, TReward -1.0, e: 0.95
episode: 12/999, TReward -1.0, e: 0.94
episode: 13/999, TReward -1.0, e: 0.94
episode: 14/999, TReward -1.0, e: 0.93
episode: 15/999, TReward -0.9974226804123711, e: 0.93
episode: 16/999, TReward -1.0, e: 0.92
episode: 17/999, TReward -1.0, e: 0.92
episode: 18/999, TReward -0.9979381443298969, e: 0.91
episode: 19/999, TReward -1.0, e: 0.91
episode: 20/999, TReward -1.0, e: 0.9
episode: 21/999, TReward -1.0, e: 0.9
episode: 22/999, TReward -0.9971590909090909, e: 0.9
episode: 23/999, TReward -1.0, e: 0.89
epis

episode: 190/999, TReward -0.9977064220183486, e: 0.39
episode: 191/999, TReward -1.0, e: 0.38
episode: 192/999, TReward -0.9975728155339806, e: 0.38
episode: 193/999, TReward -1.0, e: 0.38
episode: 194/999, TReward -0.9977678571428571, e: 0.38
episode: 195/999, TReward -1.0, e: 0.38
episode: 196/999, TReward -1.0, e: 0.37
episode: 197/999, TReward -0.9978308026030369, e: 0.37
episode: 198/999, TReward -0.9974874371859297, e: 0.37
episode: 199/999, TReward -1.0, e: 0.37
episode: 200/999, TReward -0.9979550102249489, e: 0.37
episode: 201/999, TReward -0.9975786924939467, e: 0.37
episode: 202/999, TReward -0.9967105263157895, e: 0.36
episode: 203/999, TReward -0.9979423868312757, e: 0.36
episode: 204/999, TReward -0.9972527472527473, e: 0.36
episode: 205/999, TReward -0.9969879518072289, e: 0.36
episode: 206/999, TReward -1.0, e: 0.36
episode: 207/999, TReward -1.0, e: 0.35
episode: 208/999, TReward -0.9971509971509972, e: 0.35
episode: 209/999, TReward -1.0, e: 0.35
episode: 210/999, TR

episode: 347/999, TReward -0.9970588235294118, e: 0.18
episode: 348/999, TReward -0.996309963099631, e: 0.17
episode: 349/999, TReward -0.9977876106194691, e: 0.17
episode: 350/999, TReward -0.9978586723768736, e: 0.17
episode: 351/999, TReward -0.9977116704805492, e: 0.17
episode: 352/999, TReward -1.0, e: 0.17
episode: 353/999, TReward -0.9978586723768736, e: 0.17
episode: 354/999, TReward -1.0, e: 0.17
episode: 355/999, TReward -0.9971509971509972, e: 0.17
episode: 356/999, TReward -1.0, e: 0.17
episode: 357/999, TReward -1.0, e: 0.17
episode: 358/999, TReward -1.0, e: 0.17
episode: 359/999, TReward -1.0, e: 0.17
episode: 360/999, TReward -0.9979423868312757, e: 0.16
episode: 361/999, TReward -1.0, e: 0.16
episode: 362/999, TReward -0.9974937343358395, e: 0.16
episode: 363/999, TReward -1.0, e: 0.16
episode: 364/999, TReward -0.9975247524752475, e: 0.16
episode: 365/999, TReward -0.996268656716418, e: 0.16
episode: 366/999, TReward -0.9971509971509972, e: 0.16
episode: 367/999, TRew

episode: 506/999, TReward -0.9970760233918129, e: 0.079
episode: 507/999, TReward -1.0, e: 0.079
episode: 508/999, TReward -0.9976580796252927, e: 0.078
episode: 509/999, TReward -0.9968454258675079, e: 0.078
episode: 510/999, TReward -1.0, e: 0.078
episode: 511/999, TReward -0.9973821989528796, e: 0.077
episode: 512/999, TReward -0.9974874371859297, e: 0.077
episode: 513/999, TReward -0.997716894977169, e: 0.076
episode: 514/999, TReward -0.9973890339425587, e: 0.076
episode: 515/999, TReward -0.9978858350951374, e: 0.076
episode: 516/999, TReward -0.9963898916967509, e: 0.075
--------------------Reached  episode: 517/999 TReward -0.9931972789115646, e: 0.075
episode: 518/999, TReward -0.9964912280701754, e: 0.075
episode: 519/999, TReward -0.9979253112033195, e: 0.074
episode: 520/999, TReward -0.996969696969697, e: 0.074
episode: 521/999, TReward -0.997624703087886, e: 0.073
episode: 522/999, TReward -1.0, e: 0.073
episode: 523/999, TReward -0.997275204359673, e: 0.073
episode: 524/

episode: 668/999, TReward -0.9979674796747967, e: 0.035
episode: 669/999, TReward -0.9967637540453075, e: 0.035
episode: 670/999, TReward -0.997907949790795, e: 0.035
episode: 671/999, TReward -0.9955156950672646, e: 0.035
episode: 672/999, TReward -0.9977272727272727, e: 0.034
episode: 673/999, TReward -0.9976303317535545, e: 0.034
episode: 674/999, TReward -0.9969135802469136, e: 0.034
episode: 675/999, TReward -1.0, e: 0.034
episode: 676/999, TReward -0.9975609756097561, e: 0.034
episode: 677/999, TReward -0.9974554707379135, e: 0.034
episode: 678/999, TReward -0.9975369458128078, e: 0.033
episode: 679/999, TReward -0.9961240310077519, e: 0.033
episode: 680/999, TReward -0.9971181556195965, e: 0.033
episode: 681/999, TReward -0.9971181556195965, e: 0.033
episode: 682/999, TReward -0.9968944099378882, e: 0.033
episode: 683/999, TReward -0.9973821989528796, e: 0.033
episode: 684/999, TReward -0.9964028776978417, e: 0.032
episode: 685/999, TReward -0.997979797979798, e: 0.032
episode: 

episode: 822/999, TReward -1.0, e: 0.016
episode: 823/999, TReward -0.9971098265895953, e: 0.016
episode: 824/999, TReward -0.9974683544303797, e: 0.016
episode: 825/999, TReward -0.9970238095238095, e: 0.016
episode: 826/999, TReward -1.0, e: 0.016
episode: 827/999, TReward -0.9974619289340102, e: 0.016
episode: 828/999, TReward -0.997275204359673, e: 0.016
episode: 829/999, TReward -1.0, e: 0.016
episode: 830/999, TReward -1.0, e: 0.016
episode: 831/999, TReward -1.0, e: 0.016
episode: 832/999, TReward -0.996415770609319, e: 0.015
episode: 833/999, TReward -1.0, e: 0.015
episode: 834/999, TReward -1.0, e: 0.015
episode: 835/999, TReward -0.9975247524752475, e: 0.015
episode: 836/999, TReward -0.9969230769230769, e: 0.015
episode: 837/999, TReward -1.0, e: 0.015
episode: 838/999, TReward -0.9967845659163987, e: 0.015
episode: 839/999, TReward -0.9976958525345622, e: 0.015
episode: 840/999, TReward -0.9967532467532467, e: 0.015
episode: 841/999, TReward -0.9970059880239521, e: 0.015
ep

--------------------Reached  episode: 984/999 TReward -0.9928571428571429, e: 0.01
episode: 985/999, TReward -0.9969040247678018, e: 0.01
episode: 986/999, TReward -0.9959677419354839, e: 0.01
episode: 987/999, TReward -0.9958333333333333, e: 0.01
episode: 988/999, TReward -0.9965635738831615, e: 0.01
episode: 989/999, TReward -0.9967105263157895, e: 0.01
episode: 990/999, TReward -0.9974093264248705, e: 0.01
episode: 991/999, TReward -0.9958847736625515, e: 0.01
episode: 992/999, TReward -0.9962264150943396, e: 0.01
episode: 993/999, TReward -0.9970588235294118, e: 0.01
episode: 994/999, TReward -0.9959016393442623, e: 0.01
--------------------Reached  episode: 995/999 TReward -0.9945652173913043, e: 0.01
episode: 996/999, TReward -0.9961389961389961, e: 0.01
episode: 997/999, TReward -0.995575221238938, e: 0.01
episode: 998/999, TReward -0.9966329966329966, e: 0.01
episode: 999/999, TReward -0.9959677419354839, e: 0.01
